## Load data and display statistics for the dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../archive rotten tomatoes/rotten_tomatoes_critic_reviews.csv')#, nrows = 1000)
df.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [3]:
df.describe(include='all')

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
count,1130017,1111488,1130017,1130017,1130017,824081,1130017,1064211
unique,17712,11108,2,2230,2,814,8015,949181
top,m/star_wars_the_rise_of_skywalker,Emanuel Levy,False,New York Times,Fresh,3/5,2000-01-01,Parental Content Review
freq,992,8173,841481,13293,720210,90273,48019,267


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130017 entries, 0 to 1130016
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   rotten_tomatoes_link  1130017 non-null  object
 1   critic_name           1111488 non-null  object
 2   top_critic            1130017 non-null  bool  
 3   publisher_name        1130017 non-null  object
 4   review_type           1130017 non-null  object
 5   review_score          824081 non-null   object
 6   review_date           1130017 non-null  object
 7   review_content        1064211 non-null  object
dtypes: bool(1), object(7)
memory usage: 61.4+ MB


## Cleaning data

#### remove duplicated rows (edges)

In [5]:
# Keep the last row for each unique combination of ['rotten_tomatoes_link', 'critic_name']
df.drop_duplicates(subset=['rotten_tomatoes_link', 'critic_name'], keep='last', inplace=True)
df[df.duplicated(['rotten_tomatoes_link', 'critic_name'], keep=False)].shape[0] # print duplicate_rows

0

#### critic_name

In [6]:
rows = df.shape[0]
df = df.dropna(subset=['critic_name']) # drop rows containing NaN
print(f'{rows - df.shape[0]}')

7121


#### top_critic and review_type

In [7]:
df['top_critic'] = df['top_critic'].astype(int)
df.loc[:, 'review_type'] = df['review_type'].apply(lambda x: 1 if x == 'Fresh' else 0 if x == 'Rotten' else x)
df['review_type'].value_counts()

review_type
1    630678
0    358671
Name: count, dtype: int64

#### review_score

In [8]:
# Drop NaN values. We could use review_content with NLP to impute missing values

df = df.dropna(subset=['review_score']) # drop rows containing NaN

In [9]:
print(df['review_score'].value_counts())    

review_score
3/5         78613
4/5         72927
3/4         63367
2/5         52428
2/4         41600
            ...  
8.666/10        1
0.96/10         1
7.10/10         1
3.021/10        1
9.56/10         1
Name: count, Length: 813, dtype: int64


In [10]:
def convert_review_score(value):
    ''' scale a score between 0 and 100'''
    if value is None:
        return None

    try:
        return float(value)
    except Exception:
        try:
            numerator = float(value.split('/')[0])
            divisor = float(value.split('/')[1])

            if divisor == 0:
                return None

            converted_value = round((numerator / divisor) * 100, 1)
            return converted_value
        except (ValueError, AttributeError):

            grade_mapping = {'A': 90, 'A-': 85, 'B+': 80, 'B': 75, 'B-': 70, 
                             'C+': 65, 'C': 60, 'C-': 55, 'D+': 50, 'D': 45, 'D-': 40, 'F': 0}

            if value in grade_mapping:
                return grade_mapping[value]
            else:
                return None

In [11]:
df['review_score'].isna().sum()#.sum()

0

In [12]:
df['review_score'] = df['review_score'].apply(convert_review_score)
df = df[df['review_score'] <= 100]	# also drops None
df['review_score_scaled'] = pd.cut(x=df['review_score'], bins=[0, 5, 15, 25, 35, 45, 55, 65, 75, 85, 95, 100],
					labels=[0,1,2,3,4,5,6,7,8,9,10 ] )

In [13]:
print(df['review_score_scaled'].isna().sum())
df = df.dropna(subset=['review_score_scaled']) # drop rows containing NaN
df['review_score_scaled'].isna().sum()

6082


0

In [14]:
print(df['review_score_scaled'].value_counts())

review_score_scaled
6     151097
7     140455
8     114242
4      82787
5      79878
9      57081
10     39210
2      33015
3      10652
1       4875
0       1020
Name: count, dtype: int64


In [15]:
used_features = ['review_score_scaled', 'top_critic', 'review_type']

df[used_features].isna().sum()

review_score_scaled    0
top_critic             0
review_type            0
dtype: int64

In [16]:
print(df[used_features].to_numpy().shape)
print(df[used_features].to_numpy())

(714312, 3)
[[7 0 1]
 [2 0 0]
 [7 1 1]
 ...
 [8 0 1]
 [7 0 0]
 [6 0 0]]


In [17]:
df[['rotten_tomatoes_link','critic_name']+used_features]

,rotten_tomatoes_link,critic_name,review_score_scaled,top_critic,review_type
3,m/0814255,Ben McEachen,7,0,1
6,m/0814255,Nick Schager,2,0,0
7,m/0814255,Bill Goodykoontz,7,1,1
8,m/0814255,Jordan Hoffman,7,0,1
9,m/0814255,Jim Schembri,6,1,1
...,...,...,...,...,...
1130012,m/zulu_dawn,Chuck O'Leary,4,0,0
1130013,m/zulu_dawn,Ken Hanke,7,0,1
1130014,m/zulu_dawn,Dennis Schwartz,8,0,1
1130015,m/zulu_dawn,Christopher Lloyd,7,0,0


## Save data

In [18]:
dd = df[['critic_name','rotten_tomatoes_link']+used_features]

In [19]:
dd = dd.rename({'critic_name':'Source', 'rotten_tomatoes_link':'Target'},axis=1)
dd.to_csv('../data/reviews.csv',index=False)

In [20]:
dd.loc[:1000].to_csv('../data/reviews_small.csv',index=False)